In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import glob
import mmmpy
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#from cartopy.io.img_tiles import StamenTerrain
#import pygrib
import os
#import pyart
%matplotlib inline

# Sample download MRMS directly from NCEP

In [ ]:
def download_files(input_dt, max_seconds=300):
    """
    This function takes an input datetime object, and will try to match with the closest mosaics in time
    that are available at NCEP. Note that NCEP does not archive much beyond 24 hours of data.
    
    Parameters
    ----------
    input_dt : datetime.datetime object
        input datetime object, will try to find closest file in time on NCEP server
    
    Other Parameters
    ----------------
    max_seconds : int or float
        Maximum number of seconds difference tolerated between input and selected datetimes,
        before file matching will fail
    
    Returns
    -------
    files : 1-D ndarray of strings
        Array of mosaic file names, ready for ingest into MMM-Py
    """
    baseurl = 'http://mrms.ncep.noaa.gov/data/3DReflPlus/'
    page1 = pd.read_html(baseurl)
    directories = np.array(page1[0][0][3:-1])  # May need to change indices depending on pandas version
    urllist = []
    files = []
    for i, d in enumerate(directories):
        print(baseurl + d)
        page2 = pd.read_html(baseurl + d)
        filelist = np.array(page2[0][0][3:-1])  # May need to change indices depending on pandas version
        dts = []
        for filen in filelist:
            # Will need to change in event of a name change
            dts.append(dt.datetime.strptime(filen[32:47], '%Y%m%d-%H%M%S'))
        dts = np.array(dts)
        diff = np.abs((dts - input_dt))
        if np.min(diff).total_seconds() <= max_seconds:
            urllist.append(baseurl + d + filelist[np.argmin(diff)])
            files.append(filelist[np.argmin(diff)])
    for url in urllist:
        print(url)
        #os.system('wget ' + url)
    return np.array(files)

In [ ]:
custom_dt = dt.datetime.strptime("2018-01-01_00:00:00", "%Y-%m-%d_%H:%M:%S")
files = download_files(custom_dt)

In [11]:
baseurl = 'https://noaa-mrms-pds.s3.amazonaws.com/index.html#CONUS/MergedReflectivityAtLowestAltitude_00.50/'
page1 = pd.read_html(baseurl)
page1

[Empty DataFrame
 Columns: [Object, Folder, Last Modified, Timestamp, Size]
 Index: []]

In [12]:
import boto3

ModuleNotFoundError: No module named 'boto3'